# Lab09

In [1]:
import numpy
import tensorflow
from tensorflow.keras.datasets import imdb, mnist
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import LSTM, SimpleRNN, Input
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam, RMSprop
import numpy as np
import matplotlib.pyplot as plt
# import scipy
import random
from tqdm import tqdm_notebook

from scipy import stats
# fix random seed for reproducibility
numpy.random.seed(1)

# EXERCISE 1 : MNIST GAN - Learn to generate MNIST digits

In [2]:
(X_train, Y_train), (X_test, Y_test) =  mnist.load_data()

Rescale data since we are using ReLU activations. <b>WHY ?</b>

In [3]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [4]:
z_dim = 100

#### BUILD MODEL

We are using LeakyReLU activations. 

We will build

a.) Generator
b.) Discriminator
c.) GAN

as feedforwards with multiple layers, dropout and LeakyReLU.

In [3]:
#@title
adam = Adam(lr=0.0002, beta_1=0.5)

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")


#GENERATOR
g = Sequential()
#Build your generator - noise_dim -> 256 -> 512 ->1024 ->784. LeakyRelU(0.2), adam
g.add(Dense(256, input_dim=z_dim, activation='selu'))
g.add(DefaultConv2D(filters=128))
g.add(keras.layers.MaxPooling2D(pool_size=2))
g.add(DefaultConv2D(filters=256))
g.add(keras.layers.MaxPooling2D(pool_size=2))
g.add(keras.layers.Flatten())
g.add(keras.layers.Dense(units=128, activation='relu'))
g.add(keras.layers.Dropout(0.5))
g.add(keras.layers.Dense(units=64, activation='relu'))
g.add(keras.layers.Dropout(0.5))
g.add(Dense(32, activation='sigmoid'))  # Values between 0 and 1
g.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


#DISCRIMINATOR
#Build your discriminator - 784 -> 1024 -> 512 -> 256 -> 1. LeakyRelu(0.2), adam
d = Sequential()
d.add(Dense(1024, input_dim=784, activation='selu'))
d.add(Dropout(0.3))
d.add(Dense(512, activation='selu'))
d.add(Dropout(0.3))
d.add(Dense(256, activation='selu'))
d.add(Dropout(0.3))
d.add(Dense(1, activation='sigmoid'))  # Values between 0 and 1
d.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


#GAN
d.trainable = False
inputs = Input(shape=(z_dim, ))
hidden = g(inputs)
output = d(hidden)
gan = Model(inputs, output)
gan.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

NameError: name 'Adam' is not defined

Let us write some visualization code.

In [6]:
def plot_loss(losses):
    """
    @losses.keys():
        0: loss
        1: accuracy
    """
    d_loss = [v[0] for v in losses["D"]]
    g_loss = [v[0] for v in losses["G"]]
    
    plt.figure(figsize=(10,8))
    plt.plot(d_loss, label="Discriminator loss")
    plt.plot(g_loss, label="Generator loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def plot_generated(n_ex=10, dim=(1, 10), figsize=(12, 2)):
    noise = np.random.normal(0, 1, size=(n_ex, z_dim))
    generated_images = g.predict(noise)
    generated_images = generated_images.reshape(n_ex, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

<b> TRAIN THE MODEL </b>

Generate noise, feed into generator, compare them with discriminator, train the GAN and REPEAT.

In [ ]:
losses = {"D":[], "G":[]}

def train(epochs=1, plt_frq=1, BATCH_SIZE=128):
    batchCount = int(X_train.shape[0] / BATCH_SIZE)
    print('Epochs:', epochs)
    print('Batch size:', BATCH_SIZE)
    print('Batches per epoch:', batchCount)
    
    for e in tqdm_notebook(range(1, epochs+1)):
        if e == 1 or e%plt_frq == 0:
            print('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in range(batchCount):  # tqdm_notebook(range(batchCount), leave=False):
            # Create a batch by drawing random index numbers from the training set
            image_batch = X_train[np.random.randint(0, X_train.shape[0], size=BATCH_SIZE)]
            
            # Create noise vectors for the generator
            noise = np.random.normal(0, 1, size=(BATCH_SIZE, z_dim))
            
            # Generate the images from the noise
            generated_images = g.predict(noise)
            X = np.concatenate((image_batch, generated_images))
            
            # Create Y labels similar to last exercise.
            y1 = np.zeros(2*BATCH_SIZE)
            y1[:BATCH_SIZE] =1 

            # Train gan and disrciminator similar to last exercise.
            ##YOUR CODE HERE###
            d.trainable = True
            d_loss = d.train_on_batch(X, y1)
            
            noise = np.random.normal(0, 1, size=(BATCH_SIZE, z_dim))
            y2 = np.ones(BATCH_SIZE)
            d.trainable = False
            g_loss = gan.train_on_batch(noise, y2) 

        # Only store losses from final
        losses["D"].append(d_loss)
        losses["G"].append(g_loss)

        # Update the plots
        if e == 1 or e%plt_frq == 0:
            plot_generated()
    plot_loss(losses)

In [ ]:
train(epochs=200, plt_frq=40, BATCH_SIZE=128)